In [1]:
import numpy as np
import pandas as pd
import os,sys


In [ ]:
!pip install ktrain

**Import Data**

In [8]:
df=pd.read_csv('/content/drive/My Drive/IBM data/train.csv')

**Split Data According to categories**

**Replace Empty/Nan Values With Zero**

In [9]:
df['sadness'] = df['sadness'].replace(np.nan, 0)
df['fear'] = df['fear'].replace(np.nan, 0)
df['anger'] = df['anger'].replace(np.nan, 0)
df['joy'] = df['joy'].replace(np.nan, 0)
df['confident'] = df['confident'].replace(np.nan, 0)
df['analytical'] = df['analytical'].replace(np.nan, 0)

**Data Split**

Sadness

In [10]:
from sklearn.model_selection import train_test_split

sadxtrain, sadxtest, sadytrain, sadytest = train_test_split(df['clean_text'], df['sadness'],shuffle=True)
sadxtrain=sadxtrain.values
sadytrain=sadytrain.values
sadxtest=sadxtest.values
sadytest=sadytest.values

Fear

In [11]:
fxtrain, fxtest, fytrain, fytest = train_test_split(df['clean_text'], df['fear'],shuffle=True)
fxtrain=fxtrain.values
fxtest=fxtest.values
fytrain=fytrain.values
fytest=fytest.values

Anger

In [12]:
axtrain, axtest, aytrain, aytest = train_test_split(df['clean_text'], df['anger'],shuffle=True)
axtrain=axtrain.values
axtest=axtest.values
aytrain=aytrain.values
aytest=aytest.values

Joy

In [13]:
jxtrain, jxtest, jytrain, jytest = train_test_split(df['clean_text'], df['joy'],shuffle=True)
jxtrain=jxtrain.values
jxtest=jxtest.values
jytrain=jytrain.values
jytest=jytest.values

Confident

In [14]:
cxtrain, cxtest, cytrain, cytest = train_test_split(df['clean_text'], df['confident'],shuffle=True)
cxtrain=cxtrain.values
cxtest=cxtest.values
cytrain=cytrain.values
cytest=cytest.values

Analytical

In [15]:
anlxtrain, anlxtest, anlytrain, anlytest = train_test_split(df['clean_text'], df['analytical'],shuffle=True)
anlxtrain=anlxtrain.values
anlxtest=anlxtest.values
anlytrain=anlytrain.values
anlytest=anlytest.values

In [21]:
import ktrain
from ktrain import text
MODEL_NAME = 'distilbert-base-uncased'


In [ ]:
t = text.Transformer(MODEL_NAME, maxlen=100, class_names=['1','0'])
trn = t.preprocess_train(sadxtrain, sadytrain)
val = t.preprocess_test(sadxtest, sadytest)
model = t.get_classifier()


In [ ]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [26]:
learner.fit_onecycle(8e-5, 4)



begin training using onecycle policy with max lr of 8e-05...
Train for 938 steps, validate for 59 steps
Epoch 1/4
938/938 [==============================] - 186s 198ms/step - loss: 0.3817 - accuracy: 0.8443 - val_loss: 0.3373 - val_accuracy: 0.8640
Epoch 2/4
938/938 [==============================] - 172s 183ms/step - loss: 0.2984 - accuracy: 0.8812 - val_loss: 0.3276 - val_accuracy: 0.8613
Epoch 3/4
938/938 [==============================] - 172s 183ms/step - loss: 0.1979 - accuracy: 0.9212 - val_loss: 0.3597 - val_accuracy: 0.8715
Epoch 4/4
938/938 [==============================] - 172s 183ms/step - loss: 0.0534 - accuracy: 0.9829 - val_loss: 0.4160 - val_accuracy: 0.8635


In [27]:
learner.validate(class_names=t.get_classes())

              precision    recall  f1-score   support

           1       0.92      0.92      0.92      1569
           0       0.58      0.58      0.58       306

    accuracy                           0.86      1875
   macro avg       0.75      0.75      0.75      1875
weighted avg       0.86      0.86      0.86      1875



array([[1442,  127],
       [ 129,  177]])

In [28]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [ ]:
#import json
model_json = learner.model.to_json()
with open("sadmodel.json", "w") as json_file:
    json_file.write(model_json)

learner.model.save_weights("model.h5")
print("Saved model to disk")


In [ ]:
from keras.models import load_model
newmodel=load_model('sadmodel.h5')